In [3]:
import hashlib, math
def Mod(a, b):
    return a%b

def mod_binary(k, g, p):
    y=1
    k = str(bin(k))[2:]
    for i in k:
        if i=='1':
            y = Mod(Mod(y**2, p)*g, p)
            # print(y)
        else:
            y = Mod(y**2, p)
            # print(y)
    return y

def ex_euclid(a, b):
    A = [a, b]
    X = [1, 0]
    Y = [0, 1]
    i=1
    while A[-1]!=0:
        q = A[i-1]//A[i]
        a_next = A[i-1]-A[i]*q
        x_next = X[i-1]-q*X[i]
        y_next = Y[i-1]-q*Y[i]
        A.append(a_next)
        X.append(x_next)
        Y.append(y_next)
        i+=1
    
    ans = []
    ans.append((X[i-1], Y[i-1]))
    d = A[i-1]
    x = X[i-1]
    y = Y[i-1]
    return x, y, d

def inv(a, n):
    x, y, d = ex_euclid(a, n)
    inv = x%n
    # print("inv =", inv)
    # print("inv*a mod n =", (inv*a)%n)
    return inv

# ハッシュ関数Shake128によるハッシュ値の計算
# Input: メッセージ m (通常の文字列), ベースポイントの位数 l (整数)
# Output: shake128による m のハッシュ値 (16進文字列)
def shake128(m, l):
    return hashlib.shake_128(m.encode('utf-8')).hexdigest(math.ceil(l.bit_length()/8)-1)

def dsa_sign_gen_key(p, g, l, sk):
    #Input: 法 mod：p, ベースポイント base point：g, ベースポイントの位数 base point number position：ℓ, 秘密鍵secret key：sk
    #Output: 公開鍵 public key: pk = gsk mod p
    return mod_binary(sk, g, p)

def dsa_sign_gen(m, g, l, r, p, sk):
    u = mod_binary(r, g, p) % l
    m_prime = shake128(m, l)
    v = (inv(r, l)*(int(m_prime, 16) + sk*u)) % l
    return (u, v)

def dsa_sign_verify(m, sigma, l, g, p, pk):
    u, v = sigma
    m_prime = shake128(m, l)
    v_inv = inv(v, l)
    u_prime = (mod_binary(int(m_prime, 16)*v_inv, g, p) * mod_binary(u*v_inv, pk, p)) % p % l
    if u_prime == u:
        return 'OK'
    else:
        return 'NG'

In [4]:
p4=141108755332974711606815218263958123381184588212061018448136404826965889433079453789166218230378522228564058127860367190611065605375025546257531489369344062782521806978218808940091447658298351853603237069980597505163602473095615670998464391197300372933147772009493823037167642459378452731092557170909406945309
g4=79207621787760038235763239269746451281552097558625763440050213854787240633084667257397421010854631623596917364929357681934505810865796708268321894884518347711092708958596829555918931536779252059763083320084867242870421484137196365244257388686997557137455046446997809953054632950185678637137955632299024284915
l4 = 1362115923099293242369922261305212343561846087883
sk = 370750818665451459123451714499640833062234544321
# (1)
pk = dsa_sign_gen_key(p4, g4, l4, sk)
print("pk =", pk)
# (2)
m = "123750811111151459123451714499640333335555544444"
r = 123456789123456789123456789123456789123456789123
sigma = dsa_sign_gen(m, g4, l4, r, p4, sk)
print("sigma =", sigma)
print("verify =", dsa_sign_verify(m, sigma, l4, g4, p4, pk))

pk = 116857456160049562641994494461989312437338129910496867738099983158759858021180526724378111155869568646603426364734805602803792091578036033561397592618019010094925490498520916213091171770146475950169059276842895143330427248828702351997814816800856645754804059034535942148375548680036273027009417787586210668939
sigma = (1198642514289996608651292588855800695584441096609, 203030295001677575052048798283831513552900368403)
verify = OK


In [12]:
import binascii
m = "今日の体温は36.5度です"
# print("m =", int(binascii.hexlify(m.encode('utf-8')), 16))
sk = 1033518767171415399648596791610869410243344457909
r = 123456789123456789123456789123456789123456789123
pk = dsa_sign_gen_key(p4, g4, l4, sk)
print("pk =", pk)
print("pk =", hex(pk))
sigma = dsa_sign_gen(m, g4, l4, r, p4, sk)
print("sigma =", sigma)
sigma_p = (hex(sigma[0]), hex(sigma[1]))
print("sigma =", sigma_p)

m = 404135502093172983027781195698062734278444985527097818035992618466965422489
pk = 136150644009302630913702582649620119533778702615863555732256280557674421741884034673276135347141292002359672782520907093322852774854417957045361400789785858622001356628803010317554148247555192557079140987775463521577124974604015149316864906271422014141440740595727288567824404704464590931546439601559608715228
pk = 0xc1e28c44dafdb19be3cbd11950eeb66c2f4218c9b7177c049228788f58a010d8df4d194b84f83ecaee7f2720c505555c01323bebe89ce61f40ffd23019e40eb79e09086c31e33f1be4ad67729b975522eea6b0de0c9f13cb3b26cc297e6b8909bbb05a1b7bb6faa1c0776010a8e812e771eb0523bc704209bf0b8413445767dc
sigma = (1198642514289996608651292588855800695584441096609, 1187322817516073908433816528106417933059132464126)
sigma = ('0xd1f4fd40821ef8f4629cca322b39d8214e51d9a1', '0xcff965817b3ad72f5b288d484df61a503ae39bfe')
